In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pysam
from collections import Counter
from matplotlib import cm
from matplotlib.colors import ListedColormap

import io
from pysam import VariantFile

from sklearn.metrics import precision_score, recall_score, average_precision_score, precision_recall_curve

In [ ]:
tab20 = cm.get_cmap('tab10', 8)
newcmap_list = []
for i in range(4):
    newcmap_list.append(tab20.colors[i])
for i in range(4):
    newcmap_list.append(np.array(list(tab20.colors[i][:-1]) +[0.3]))

newcmap = ListedColormap(newcmap_list, name='newcmap')
color_list = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple']

# Dilution effect on 809_110914 VAF distribution

## Load data

In [ ]:
# vcf vaf files
vcf_pd_0 = pd.read_csv("../data/2015-07-31_CRC-809_110914-filter-1-0/CRC-809_110914-filter-1-0-ensemble-annotated_outputvaf.vcf.frq", sep='\t')
vcf_pd_0 = vcf_pd_0.reset_index()
vcf_pd_0.columns = ['CHROM', 'POS', 'N_ALLELES', 'N_CHR', 'ALLELE1_FREQ', 'ALLELE2_FREQ']
vcf_pd_0[['main allele','freq main allele']] =  pd.DataFrame(vcf_pd_0.ALLELE1_FREQ.str.split(':',1).tolist(), columns = ['main allele','freq main allele'])
vcf_pd_0['freq main allele'] = vcf_pd_0['freq main allele'].astype(float)

vcf_pd_1 = pd.read_csv("../data/2015-07-31_CRC-809_110914-filter-05-05/CRC-809_110914-filter-05-05-ensemble-annotated_outputvaf.vcf.frq", sep='\t')
vcf_pd_1 = vcf_pd_1.reset_index()
vcf_pd_1.columns = ['CHROM', 'POS', 'N_ALLELES', 'N_CHR', 'ALLELE1_FREQ', 'ALLELE2_FREQ']
vcf_pd_1[['main allele','freq main allele']] =  pd.DataFrame(vcf_pd_1.ALLELE1_FREQ.str.split(':',1).tolist(), columns = ['main allele','freq main allele'])
vcf_pd_1['freq main allele'] = vcf_pd_1['freq main allele'].astype(float)

vcf_pd_2 = pd.read_csv("../data/2015-07-31_CRC-809_110914-filter-005-095/CRC-809_110914-filter-005-095-ensemble-annotated_outputvaf.vcf.frq", sep='\t')
vcf_pd_2 = vcf_pd_2.reset_index()
vcf_pd_2.columns = ['CHROM', 'POS', 'N_ALLELES', 'N_CHR', 'ALLELE1_FREQ', 'ALLELE2_FREQ']
vcf_pd_2[['main allele','freq main allele']] =  pd.DataFrame(vcf_pd_2.ALLELE1_FREQ.str.split(':',1).tolist(), columns = ['main allele','freq main allele'])
vcf_pd_2['freq main allele'] = vcf_pd_2['freq main allele'].astype(float)

vcf_pd_3 = pd.read_csv("../data/2015-07-31_CRC-809_110914-filter-001-099/CRC-809_110914-filter-001-099-ensemble-annotated_outputvaf.vcf.frq", sep='\t')
vcf_pd_3 = vcf_pd_3.reset_index()
vcf_pd_3.columns = ['CHROM', 'POS', 'N_ALLELES', 'N_CHR', 'ALLELE1_FREQ', 'ALLELE2_FREQ']
vcf_pd_3[['main allele','freq main allele']] =  pd.DataFrame(vcf_pd_3.ALLELE1_FREQ.str.split(':',1).tolist(), columns = ['main allele','freq main allele'])
vcf_pd_3['freq main allele'] = vcf_pd_3['freq main allele'].astype(float)

# tumor fraction estimation
samples_tf = {
    'sample_0': 0.47,
    'sample_1': float(list(pd.read_csv("../data/2015-07-31_CRC-809_110914-filter-05-05/estimated_tf.txt").columns)[0]),
    'sample_2': float(list(pd.read_csv("../data/2015-07-31_CRC-809_110914-filter-005-095/estimated_tf.txt").columns)[0]),
    'sample_3': float(list(pd.read_csv("../data/2015-07-31_CRC-809_110914-filter-001-099/estimated_tf.txt").columns)[0]),
}

print(samples_tf)

## Plot VAF

In [ ]:
for i, vcf_pd in enumerate([vcf_pd_0, vcf_pd_1, vcf_pd_2, vcf_pd_3]):
    plt.figure()
    plt.hist(vcf_pd['freq main allele'])
    plt.title('sample_'+str(i)+', tf={0:.3f}%'.format(100*samples_tf['sample_'+str(i)]))

- Benchmarking results for germline SNVs
- Benchmarking results for somatic SNVs on exome data.
- averaged over the four replicates

- add fake mutations on healthy mixtures